In [1]:
# imports
import os
import pandas as pd
from skimage.measure import label,regionprops
from skimage.morphology import binary_opening, erosion, dilation, ball
from skimage.segmentation import watershed
from scipy import ndimage
from sklearn.mixture import BayesianGaussianMixture
from skimage.morphology import opening, closing, ball
from skimage.filters import gaussian
import numpy as np
import nrrd
from skimage.segmentation import morphological_geodesic_active_contour as gac
from skimage.segmentation import inverse_gaussian_gradient as igg
from scipy.ndimage.morphology import binary_fill_holes
from scipy.stats import entropy
# from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

In [2]:
''' 
SUPPLEMENTARY FUNCTION : createSubVolumes
This function isolates the sub-volume of the image that we are interested in. 
This way, we can perform operations only on the pixels containing lesion and surrounding pixels (executes faster).
    
    INPUT:
            image         - the original CT volume;
            image_dict    - dictionary with CT metadata;
            mask          - the original mask with radiologist-defined ROIs;
            lprop         - regionprops object for the user-defined start point.
            
    OUTPUT:
            cylinder_subV - cylinder mask, which occupies same space as both image and mask sub-volumes;
            image_subV    - image sub-volume, which contains lesion plus surrounding voxels;
            mask_subV     - ground truth labels, which occupies same space as both image and cylinder sub-volumes;
            R             - cylinder radius, for downstream calculations.
            
'''

def createSubVolumes(image,image_dict,mask,lprop):
    
    # construct cylinder_image
    bbox = lprop.bbox  
    bbox_dims = [bbox[3]-bbox[0],bbox[4]-bbox[1],bbox[-1]-bbox[2]]
    
    # take a circle with radius larger than bounding box
#   R = bbox_dims[0]/2 + bbox_dims[1]/2  
    R = max(bbox_dims)*5/8       
    
    # isolate the centroid of the nodule -- this will be our preliminary marker 
    j,i,k = [round(i) for i in lprop.centroid] 

    circle_mask = np.zeros((image.shape[0],image.shape[1]),dtype=bool)
    num_rows,num_cols = circle_mask.shape

    row,col = np.meshgrid(range(num_rows),range(num_cols))
    circle_mask[((row-i)**2+(col-j)**2)<R**2] = True
            
    # now we have the initial contour (that will theoretically be user-defined for the clinical dataset)
    # next step: extend the circle in both superior and inferior directions to obtain a cylinder

    # determine the number of slices required in the z direction for full coverage
    num_slices = int(np.ceil((float(image_dict['pixel_spacing']) * R) / float(image_dict['slice_thickness'])/2))

    cylinder_image = np.zeros((image.shape[0],image.shape[1],image.shape[2]),dtype=bool)

    for i in range(k-num_slices,k+num_slices+1):
        cylinder_image[:,:,i] = circle_mask
                
    # label the mask into connected regions
    mask_labels = label(cylinder_image)
    mask_props = regionprops(mask_labels)

    coords = mask_props[0].bbox

    cylinder_subV = mask_props[0].image
    image_subV = image[coords[0]:coords[3],coords[1]:coords[4],coords[2]:coords[-1]]
    mask_subV = mask[coords[0]:coords[3],coords[1]:coords[4],coords[2]:coords[-1]]
    
    return cylinder_subV,image_subV,mask_subV,R

In [3]:
''' 
SUPPLEMENTARY FUNCTION : createSubVolumesClinical
This function isolates the sub-volume of the image that we are interested in. 
This way, we can perform operations only on the pixels containing lesion and surrounding pixels (executes faster).
    
    INPUT:
            image         - the original CT volume;
            image_dict    - dictionary with CT metadata;
            mask          - the original mask with radiologist-defined ROIs;
            lprop         - regionprops object for the user-defined start point.
            
    OUTPUT:
            image_subV    - image sub-volume, which contains lesion plus surrounding voxels;
            mask_subV     - ground truth labels, which occupies same space as both image and cylinder sub-volumes.
            
'''

def createSubVolumesClinical(image,mask,lprop):
    
    # get the bounding box for the user-defined mask in image coordinates
    coords = lprop.bbox 
    row = 3
    slc = 1
    
    # crop the image and mask to the expanded bounding box
    image_subV = image[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    mask_subV = mask[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    
    return image_subV,mask_subV

In [4]:
''' 
SUPPLEMENTARY FUNCTION : determineThreshold
This function determines a threshold separating candidate lesion voxels from surrounding lung paremchyma. 
It leverages the BayesianGaussianMixture functionality from sklearn. 
Since the pre-defined sub-volume contains both lesion and lung parenchyma pixels, density distribution 
of the pixels could be modeled by two Gaussian distributions: P(x|lesion) and P(x|parenchyma), 
where x was the pixel density. The mean values and variations of the two Gaussian distributions were 
then estimated by the expectation-maximization method.
    
    INPUT:
            image_subV - the CT sub-volume;
            mask_subV  - mask occupying the same space as the image_subV.
            
    OUTPUT:
            threshold     - floating point threshold (anything above this threshold is a candidate lesion voxel)
                     
'''

def determineThreshold(image_subV,mask_subV):
    
    # isolate the intensities within the mask
    intensities = image_subV[mask_subV]

    hist, bin_edges = np.histogram(intensities, bins=60)
    bin_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])

    classif = BayesianGaussianMixture(n_components=2)
    classif.fit(intensities.reshape(-1,1))

    return np.mean(classif.means_)

In [5]:
# lists of image/mask files
img_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images'
msk_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Masks'

all_images = sorted([os.path.join(img_path,f) for f in os.listdir(img_path) if 'nrrd' in f])
all_masks = sorted([os.path.join(msk_path,f) for f in os.listdir(msk_path) if 'nrrd' in f])

<div class="alert alert-block alert-info">
<b>NOTE:</b>
    
The `all_images` and `all_masks` variables contain pointers to individual files. The files in these folders are not separated by time point. Take the first four pointers in the `all_images` variable as an example:
<br>   
`./Images/112001_BL_image.nrrd`,<br>
`./Images/112001_C2_image.nrrd`,<br>
`./Images/112002_BL_image.nrrd`,<br>
`./Images/112002_C2_image.nrrd`,...<br>
<br>   
The first six digits is the unique subject identifier, and the two letters that follow encode the timepoint, where **BL** is the baseline or first measurement and **C2** is the measurement after 2 cycles of chemotherapy or second measurement.
</div>

In [ ]:
# first pass of segmentation pipeline
def segPipeline(imgList,mskList):
    
    # import the trial arm key -- the arm that received the hypoxia drug is encoded as 1
    key = pd.read_csv('./trial_arm_key.csv')
    
    # initialize lists for segmentation accuracy
    usubjid = []
    timepoint = []
    trial_arm = []
    lesion_index = []
    lesion_volume = []
    
    for i in range(len(imgList)):
        
        # read image and corresponding mask
        img_V,img_d = nrrd.read(imgList[i])
        msk_V,msk_d = nrrd.read(mskList[i])
        
        # calculate image and mask voxel volumes
        patient_id,time_id,fileID = os.path.basename(imgList[i]).split('_')
        msk_voxel_volume = msk_d['space directions'][0][0]**2 * msk_d['space directions'][-1][-1] # in mm^3
        img_voxel_volume = img_d['space directions'][0][0]**2 * img_d['space directions'][-1][-1] # in mm^3
        
        print('mask voxel volume: ',msk_voxel_volume)
        print('image voxel volume: ',img_voxel_volume)
        
        # if the image and mask do not have the same voxel volume, do not process
        if abs(msk_voxel_volume - img_voxel_volume) > 0.0001:
            print('ERROR, {}: image and mask must have same voxel volumes'.format(imgList[i]))
            continue
            
        # label the mask into connected regions
        lesion_labels,num_lesions = label(msk_V,return_num=True)
        lesion_props = regionprops(lesion_labels)

        # track lesion number
        lesion_count = 0
        
        # for every lesion in the mask
        for obj in lesion_props:
            
            usubjid.append(patient_id)
            timepoint.append(time_id)
            trial_arm.append(key.loc[key['USUBJID'] == int(patient_id),'ARM'].item())
            lesion_index.append(lesion_count)
            lesion_count += 1
            
            # create the sub-volume of interest within the CT volume
            img_subV,msk_subV = createSubVolumesClinical(img_V,msk_V,obj)
            
            # calculate threshold using EM
            threshold = determineThreshold(img_subV,msk_subV)
            msk_subV = msk_subV > 0
            
            # label the cylinder sub-volume into connected regions
            msk_subV_labels = label(msk_subV)
            msk_subV_props = regionprops(msk_subV_labels)
            
            # get the dimensions of the user-defined mask
            bbox = msk_subV_props[0].bbox  
            bbox_dims = [bbox[3]-bbox[0],bbox[4]-bbox[1],bbox[-1]-bbox[2]]
            R = max(bbox_dims) / 2 

            # coordinates for the user-defined lesion centroid
            msk_j,msk_i,msk_k = [round(i) for i in msk_subV_props[0].centroid]

            binary_img = np.logical_and(np.logical_and(img_subV > threshold,img_subV > -850),img_subV < 200)
            binary_img[~msk_subV] = False
            binary_img_centroid = closing(opening(binary_img,ball(radius=int(0.1*R))),ball(radius=int(0.1*R)))
            
            # construct a marker image for the watershed
            marker_image = np.zeros((img_subV.shape[0],img_subV.shape[1],img_subV.shape[2]),dtype=np.uint8)
            marker_image[~binary_img] = 2         # background/lung parenchyma voxels
            marker_image[msk_j,msk_i,msk_k] = 1   # user-defined lesion centroid

            # denoise image sub-volume
            denoised = gaussian(img_subV,multichannel=False)

            # try some funky stuff
            max_image = dilation(denoised,ball(2))
            min_image = erosion(denoised,ball(2))

            gradient_image = max_image - min_image
            gradient_image[~msk_subV] = np.min(gradient_image)

            # create distance matrix for the 3D bowl function
            row,col,slc = np.meshgrid(range(gradient_image.shape[1]),range(gradient_image.shape[0]),range(gradient_image.shape[2]))
            dist_matrix = np.sqrt((round(msk_j) - row)**2 + (round(msk_i) - col)**2 + ((round(msk_k) - slc) * msk_d['space directions'][-1][-1])**2)
            dist_matrix[dist_matrix>=R] = R
            dist_matrix = dist_matrix / R
            
            # modify the gradient image for watershed
            mod_gradient = gradient_image * dist_matrix
            
            # perform watershed segmentation
            water_initial = ~watershed(gradient_image,marker_image,connectivity=2)
            water_initial_mod = ~watershed(mod_gradient, marker_image,connectivity=2)
            
            # label the watershed mask into connected regions
            water_labels = label(water_initial_mod,background = np.min(water_initial_mod))
            water_props = regionprops(water_labels)
            
            # find the smallest region
            water_areas = [water_props[i].area for i in range(len(water_props))]
            ind = np.where(water_areas == np.min(water_areas))

            # create the mask
            water_mask = water_labels == water_props[ind[0][0]].labelgit
            
            # FINALLY this initial segmentation is fed into the active contours function for further refinement
            refined_mask = gac(igg(img_subV), iterations = 1, init_level_set=water_mask, threshold = 0.5)
            
            # fill any holes in the final mask result (unlikely, but you never know)
            refined_mask = binary_fill_holes(refined_mask)
            
            lesion_volume.append(sum(water_mask) * msk_voxel_volume)
#             extra_stats = AEFunction(img_subV,water_mask)  # could be refined_mask

            output_dict = { 'USUBJID' : usubjid,         # patient identifier (6-digit number)
                            'TIMEPT'  : timepoint,       # timepoint identifier (2-letter code -- BL baseline, C2 cycle 2)
                            'ARM'     : trial_arm,       # trial arm (0 control group, 1 experimental group)
                            'LSNIND'  : lesion_index,    # lesion index (for patients with more than 1 lesion)
                            'LSNVOL'  : lesion_volume}   # lesion volume (in mm^3)
    
    
    return pd.DataFrame(output_dict)
        

# run the pipeline
df = segPipeline(all_images,all_masks)

mask voxel volume:  3.5266561280000017
image voxel volume:  3.8146777343999974
ERROR, /Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images/112001_BL_image.nrrd: image and mask must have same voxel volumes
mask voxel volume:  3.5266561280000017
image voxel volume:  3.526656128000001
bool
bool
mask voxel volume:  2.3807255616000003
image voxel volume:  2.3807255616000003
bool
bool
bool
bool
bool
bool
mask voxel volume:  4.768347168
image voxel volume:  4.768347168
bool
bool
bool
bool
bool
bool
mask voxel volume:  1.1718750000000007
image voxel volume:  1.171875
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool


/Users/EL-CAPITAN-2016/anaconda2/envs/spyder/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


bool
bool
mask voxel volume:  1.1718750000000007
image voxel volume:  1.171875
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
mask voxel volume:  2.1879480000000004
image voxel volume:  2.1879480000000004
bool
bool
mask voxel volume:  2.1879480000000004
image voxel volume:  2.1879480000000004
bool
bool
mask voxel volume:  1.232643
image voxel volume:  1.232643
bool
bool
bool
bool
mask voxel volume:  1.5582254700000002
image voxel volume:  1.5582254700000002
bool
bool
bool
bool
mask voxel volume:  1.6525379883000002
image voxel volume:  1.6525379883000002


/Users/EL-CAPITAN-2016/anaconda2/envs/spyder/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


bool
bool
bool
bool
mask voxel volume:  1.6525379883000002
image voxel volume:  1.6525379883000002
bool
bool
bool
bool
mask voxel volume:  1.6525379883000002
image voxel volume:  1.6525379883000002
bool
bool
mask voxel volume:  1.6525379883000002
image voxel volume:  1.6525379883000002
bool
bool
mask voxel volume:  2.6697842688
image voxel volume:  2.6697842688
bool
bool
mask voxel volume:  2.2662130187999985
image voxel volume:  2.2662130187999985
bool
bool
mask voxel volume:  2.7253132031999994
image voxel volume:  2.7253132031999994


In [ ]:
all_IoU_AC = (np.array(IoU_actC) + np.array(IoU_backA))/2
np.mean(all_IoU_AC)

In [ ]:
all_IoU_WS = (np.array(IoU_water) + np.array(IoU_backW))/2
np.mean(all_IoU_WS)

In [ ]:
from scipy.stats.stats import pearsonr

pearsonr(mean_intensities,IoU_actC)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(all_IoU_WS)

In [ ]:
plt.scatter(entropy_gradient,all_IoU_WS)
plt.xlabel('Avg. Intensity of Voxels > Bayesian Threshold')
plt.ylabel('Avg. IoU')

In [ ]:
# ALTERNATIVE EVALUATION METRIC : DICE COEFFICIENT
# seg = water_mask
# dice = np.sum(seg[msk_subV_true==True])*2.0 / (np.sum(seg) + np.sum(msk_subV_true))
# dice
mean_intensities_gradient

In [ ]:
mean_intensities

In [ ]:
np.std(entropy_gradient)

In [ ]:
np.mean(entropy_gradient) - 2 * np.std(entropy_gradient)

In [ ]:
dice_water

In [ ]:
HU = []
for i in range(len(dice_water)):
    if dice_water[i]<0.5:
        HU.append(mean_intensities[i])

In [ ]:
HU